##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Default title text
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# TF-Hub generative image model.

This Colab demonstrates use of a TF-Hub module based on a GAN (generative adversarial network). The module maps from N-dimensional vectors (latent space) to RGB images.

Two examples are provided:
* Mapping from latent space to images,
* Fixing an image and finding a latent vector that maps to an image that is the closest possible to the fixed one.

In [0]:
# Install the latest Tensorflow version.
!pip install "tensorflow>=1.7.0rc0"
# Install TF-Hub.
!pip install -I https://storage.googleapis.com/tfhub-test-modules/tensorflow_hub-0.9.0-py2.py3-none-any.whl
# Install imageio for creating animations.  
!pip install imageio

In [0]:
#@title Imports and function definitions

import imageio
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import time
from IPython import display

# We could retrieve this value from module.get_input_shapes() if we didn't know
# beforehand which module we will be using.
latent_dim = 512


# Interpolates between two vectors that are non-zero and don't lie on the same
# line. First normalizes v2 to have the same norm as v1. Then interpolates
# between the two vectors on the hypersphere.
def interpolate_hypersphere(v1, v2, num_steps):
  v1_norm = tf.norm(v1)
  v2_norm = tf.norm(v2)
  v2_normalized = v2 * (v1_norm / v2_norm)

  vectors = []
  for step in range(num_steps):
    interpolated = v1 + (v2_normalized - v1) * step / (num_steps - 1)
    interpolated_norm = tf.norm(interpolated)
    interpolated_normalized = interpolated * (v1_norm / interpolated_norm)
    vectors.append(interpolated_normalized)
  return tf.stack(vectors)


# Given a set of images, show an animation.
def animate(images):
  imageio.mimsave('./animation.gif', images)
  with open('./animation.gif','rb') as f:
      display.display(display.Image(data=f.read(), height=300))


# Simple way to display an image.
def display_image(image):
  plt.figure()
  plt.axis("off")
  plt.imshow(image)


# Display multiple images in the same figure.
def display_images(images, captions=None):
  num_horizontally = 5
  f, axes = plt.subplots(
      len(images) / num_horizontally, num_horizontally, figsize=(20, 20))
  for i in range(len(images)):
    axes[i / num_horizontally, i % num_horizontally].axis("off")
    if captions is not None:
      axes[i / num_horizontally, i % num_horizontally].text(0, -3, captions[i])
    axes[i / num_horizontally, i % num_horizontally].imshow(images[i])
  f.tight_layout()

tf.logging.set_verbosity(tf.logging.ERROR)

## Latent space interpolation

### Random vectors

Latent space interpolation between two randomly initialized vectors. We will use a TF-Hub module that contains pre-trained Progressive GANs based on [
Progressive Growing of GANs for Improved Quality, Stability, and Variation](https://arxiv.org/abs/1710.10196).

In [0]:
def interpolate_between_vectors():
  with tf.Graph().as_default():
    module = hub.Module("https://tfhub.dev/google/progan-128/1")

    # Change the seed to get different random vectors.
    v1 = tf.random_normal([latent_dim], seed=1)
    v2 = tf.random_normal([latent_dim], seed=2)
    
    # Creates a tensor with 50 steps of interpolation between v1 and v2.
    vectors = interpolate_hypersphere(v1, v2, 50)

    # Uses module to generate images from the latent space.
    interpolated_images = module(vectors)

    with tf.Session() as session:
      session.run(tf.global_variables_initializer())
      interpolated_images_out = session.run(interpolated_images)

    animate(interpolated_images_out)

interpolate_between_vectors()

## Finding closest vector in latent space
First fix a target image from the image space of the module.

In [0]:
def get_module_space_image():
  with tf.Graph().as_default():
    module = hub.Module("https://tfhub.dev/google/progan-128/1")
    vector = tf.random_normal([1, latent_dim], seed=10)
    images = module(vector)

    with tf.Session() as session:
      session.run(tf.global_variables_initializer())
      image_out = session.run(images)[0]
  return image_out


target_image = get_module_space_image()
display_image(target_image)

Now let's define a loss as a distance between the fixed image and image represented by our latent space variable. Perform a gradient descent on this loss with respect to the variable.

In [0]:
def find_closest_latent_vector(num_optimization_steps):
  images = []
  losses = []
  with tf.Graph().as_default():
    module = hub.Module("https://tfhub.dev/google/progan-128/1")

    initial_vector = tf.random_normal([1, latent_dim], seed=1)

    vector = tf.get_variable("vector", initializer=initial_vector)
    image = module(vector)

    loss = tf.reduce_sum(
        tf.losses.absolute_difference(image[0], target_image))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.3)
    train = optimizer.minimize(loss)

    with tf.Session() as session:
      session.run(tf.global_variables_initializer())
      for _ in range(num_optimization_steps):
        _, loss_out, im_out = session.run([train, loss, image])
        images.append(im_out[0])
        losses.append(loss_out)
        print loss_out
    return images, losses


result = find_closest_latent_vector(num_optimization_steps=20)
display_images(result[0], [("Loss: %.2f" % loss) for loss in result[1]])

### Playing with the above example
If image is from the module space, the descent is quick and converges to a reasonable sample. Try out descending to an image that is **not from the module space**.
How to make it descend faster and to a more realistic image? 
* different loss, e.g. quadratic,
* different regularizer,
* initialize from a random vector in multiple runs,
* etc.
